In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 4 # piminus, electron

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-50M_piminus_embed_single_new_04-10_v0.root","my-50M_electron_embed_single_new_04-10_v0.root","my-100M_piminus_single_hagedorn_04-5_v0.root"\
            ,"my-100M_piminus_embed_single_new_04-10_v0.root","my-50M_ccbar_pythia_v0.root"]
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron','pionminus1','pionminus2','eccbar']#output
filename=outfilenames[iOption0]
part_ids=[9,3,9,9,3]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]
N_start, N_end = 0e6, 100e6

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

49819168
Nevent: 1, centrality: 3.0, x: 0.3450630009174347, Ntracks: 3
pt:  0.194 , e/p:  1.255 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.784 , id:  positron
pt:  0.773 , e/p:  0.087 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.557 , id:  electron
pt:  0.293 , e/p:  0.356 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.859 , id:  electron
1.7879180908203125
0.8795163035392761
Nevent: 2, centrality: 23.0, x: 0.3378390073776245, Ntracks: 1
pt:  0.413 , e/p:  2.213 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.0 , id:  electron
0.4659356474876404
0.4870540201663971
Nevent: 3, centrality: 83.0, x: 0.33152198791503906, Ntracks: 1
pt:  0.704 , e/p:  1.029 , n0:  5 , disp:  1.47 , chi2/npe0:  1.7 , prob:  0.971 , id:  electron
0.7483736276626587
1.2160805463790894
Nevent: 6, centrality: 57.0, x: 0.336342990398407, Ntracks: 1
pt:  0.304 , e/p:  1.443 , n0:  3 , disp:  6.47 , chi2/npe0:  12.98 , prob:  0.179 , id:  positron
0.74419587850570

In [8]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0

for ievent in range(int(N_start), min(int(N_end),N_real_ev)):
    if (ievent - N_start) % 1e5 == 0: print("N of events:", ievent - N_start)
    if good_event_counter>100000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.5 or myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.5 or myevent.GetEntry(ipart).GetN0()>=0):
            data_array.append([int(myevent.GetCentrality()),myevent.GetEntry(ipart).GetZDC(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),min(myevent.GetEntry(ipart).GetChi2(), myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-6)),\
                            myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetDisp()**2])


data = pd.DataFrame(data_array, columns=('centrality','zdc','pt', 'e/p', 'n0', 'disp', 'chi2/npe0', 'npe0', 'prob', 'disp2'))

49819168
N of events: 0.0
N of events: 100000.0
N of events: 200000.0
N of events: 300000.0
N of events: 400000.0
N of events: 500000.0
N of events: 600000.0
N of events: 700000.0
N of events: 800000.0
N of events: 900000.0
N of events: 1000000.0
N of events: 1100000.0
N of events: 1200000.0
N of events: 1300000.0
N of events: 1400000.0
N of events: 1500000.0
N of events: 1600000.0
N of events: 1700000.0
N of events: 1800000.0
N of events: 1900000.0
N of events: 2000000.0
N of events: 2100000.0
N of events: 2200000.0
N of events: 2300000.0
N of events: 2400000.0
N of events: 2500000.0
N of events: 2600000.0
N of events: 2700000.0
N of events: 2800000.0
N of events: 2900000.0
N of events: 3000000.0
N of events: 3100000.0
N of events: 3200000.0
N of events: 3300000.0
N of events: 3400000.0
N of events: 3500000.0
N of events: 3600000.0
N of events: 3700000.0
N of events: 3800000.0
N of events: 3900000.0
N of events: 4000000.0
N of events: 4100000.0
N of events: 4200000.0
N of events: 4300

In [9]:
data.head()

,centrality,zdc,pt,e/p,n0,disp,chi2/npe0,npe0,prob,disp2
0,23,-4.785156,0.412691,2.213105,-9999,-9999.000000,-9999.000000,-9999,0.000061,9.998000e+07
1,83,-55.062500,0.703862,1.029354,5,1.470933,1.703081,20,0.970745,2.163643e+00
2,67,78.687500,0.979915,0.407967,5,0.628469,2.150390,16,0.333516,3.949738e-01
3,89,-11.390625,0.877127,1.068244,4,2.035790,8.758062,8,0.986903,4.144442e+00
4,17,61.093750,0.289958,0.551446,-9999,-9999.000000,-9999.000000,-9999,0.346589,9.998000e+07


In [10]:
print(data.size, data.shape, good_event_counter)

99230210 (9923021, 10) 9812058


In [11]:
filepath = '/home/yoren/bnl/PHENIX/ee/ML/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [12]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)